In [1]:
move_dict = {
    (0, 1): '>',
    (1, 0): 'v',
    (0, -1): '<',
    (-1, 0): '^',
}
inv_move_dict = {v: k for k, v in move_dict.items()}

num_keypad = [['7', '8', '9'], ['4', '5', '6'], ['1', '2', '3'], ['_', '0', 'A']]
num_keypad_map = {num_keypad[i][j]: (i, j) for i in range(4) for j in range(3)}

directional_keypad = [['_', '^', 'A'], ['<', 'v', '>']]
directional_keypad_map = {directional_keypad[i][j]: (i, j) for i in range(2) for j in range(3)}

assert num_keypad_map['0'] == (3, 1), num_keypad_map
assert num_keypad_map['_'] == (3, 0), num_keypad_map
assert num_keypad_map['A'] == (3, 2), num_keypad_map
assert directional_keypad_map['^'] == (0, 1), directional_keypad_map
assert directional_keypad_map['A'] == (0, 2), directional_keypad_map

In [2]:
class Keypad():
    def out_of_bounds(self, i, j, matrix):
        return i < 0 or j < 0 or i >= len(matrix) or j >= len(matrix[0])

    def __init__(self, child_keypad: list[list[str]], parent_keypad: list[list[str]], name: str):
        self.child_keypad = child_keypad
        self.child_keypad_map = {child_keypad[i][j]: (i, j) for i in range(len(child_keypad)) for j in range(len(child_keypad[0]))}
        self.parent_keypad = parent_keypad
        self.parent_keypad_map = {parent_keypad[i][j]: (i, j) for i in range(len(parent_keypad)) for j in range(len(parent_keypad[0]))}
        self.current_position = self.child_keypad_map['A']
        self.name = name
        self.init_move_map()

    def init_move_map(self):
        # for every pair of keys, find the shortest path between them
        self.move_map = {}
        for key1 in self.child_keypad_map:
            for key2 in self.child_keypad_map:
                self.move_map[(key1, key2)] = self.find_path(key1, key2)

        # we manually add some special cases to reduce the number of turns
        if self.child_keypad[0][0] == '7':
            self.move_map[('7', '0')] = ['>', 'v', 'v', 'v', 'A']
            self.move_map[('7', 'A')] = ['>', '>', 'v', 'v', 'v', 'A']
            self.move_map[('4', '0')] = ['>', 'v', 'v', 'A']
            self.move_map[('4', 'A')] = ['>', '>', 'v', 'v', 'A']
            self.move_map[('1', 'A')] = ['>', '>', 'v', 'A']
        else:
            self.move_map[('<', 'A')] = ['>', '>', '^', 'A']

    def press(self, key):
        moves = self.move_map[(self.child_keypad[self.current_position[0]][self.current_position[1]], key)]
        if moves == None:
            moves = ['A']
        self.current_position = self.child_keypad_map[key]
        return moves
    
    def reverse_presses(self, keys):
        moves = []
        pos = self.child_keypad_map['A']
        for key in keys:
            # here you are given key presses (<, >, ^, v, A) and you need to translate them to the parent keypad
            if key == 'A':
                moves += self.child_keypad[pos[0]][pos[1]]
            else:
                inv = inv_move_dict[key]
                pos = pos[0] + inv[0], pos[1] + inv[1]

        return moves

    def find_path(self, key1, key2):
        if key1 == key2:
            return None

        if key1 == '_' or key2 == '_':
            return None
        
        x1, y1 = self.child_keypad_map[key1]
        x2, y2 = self.child_keypad_map[key2]

        path: list[tuple[int, int]] = [(x1, y1)]

        while (x1, y1) != (x2, y2):
            if x1 < x2 and not self.child_keypad[x1+1][y1] == '_':
                x1 += 1
            elif x1 > x2 and not self.child_keypad[x1-1][y1] == '_':
                x1 -= 1
            elif y1 < y2 and not self.child_keypad[x1][y1+1] == '_':
                y1 += 1
            elif y1 > y2 and not self.child_keypad[x1][y1-1] == '_':
                y1 -= 1
            else:
                raise Exception(f'No path between {key1} and {key2}')
            
            path.append((x1, y1))

        # convert path to directional_keypad presses to list of moves
        # by subtracting the start position from the target position
        moves = []
        for i in range(1, len(path)):
            x1, y1 = path[i-1]
            x2, y2 = path[i]
            moves.append((x2-x1, y2-y1))

        return [move_dict[move] for move in moves] + ['A']

In [3]:
numpad = Keypad(num_keypad, directional_keypad, 'numpad')
moves = numpad.press('0')
assert moves == ['<', 'A'], moves
assert numpad.current_position == (3, 1), numpad.current_position
moves = numpad.press('2')
assert moves == ['^', 'A'], moves
assert numpad.current_position == (2, 1)

dirpad = Keypad(directional_keypad, directional_keypad, 'dirpad')
moves = dirpad.press('<')
assert moves == ['v', '<', '<', 'A'], moves
assert dirpad.current_position == (1, 0), dirpad.current_position

numpad = Keypad(num_keypad, directional_keypad, 'numpad')
numpad.current_position = (3, 1)
moves = numpad.press('7')
assert moves == ['^', '^', '^', '<', 'A'], moves

dirpad = Keypad(directional_keypad, num_keypad, 'dirpad')
dirpad.current_position = (1, 0)
moves = dirpad.press('A')
assert moves == ['>', '>', '^', 'A'], moves

numpad = Keypad(num_keypad, directional_keypad, 'numpad')
moves = numpad.press('7')
moves += numpad.press('1')
moves += numpad.press('6')
moves += numpad.press('0')
assert moves == ['^', '^', '^', '<', '<', 'A', 'v', 'v', 'A', '^', '>', '>', 'A', 'v', 'v', '<', 'A'], moves
rev = numpad.reverse_presses(moves)
assert rev == ['7', '1', '6', '0'], rev

In [4]:
doorpad = Keypad(num_keypad, directional_keypad, 'doorpad')

sequence = "029A"
moves = [doorpad.press(target) for target in sequence]
moves = [move for sublist in moves for move in sublist]

assert "".join(moves) == '<A^A^^>AvvvA', "".join(moves)
rev = doorpad.reverse_presses(moves)
assert rev == list(sequence), rev

In [5]:
doorpad = Keypad(num_keypad, directional_keypad, 'doorpad')
doorpadbot = Keypad(directional_keypad, directional_keypad, 'doorpadbotpad')

# pipe the presses from the human to the door
sequence = "029A"

doorpad_moves = [doorpad.press(target) for target in sequence]
doorpad_moves = [move for sublist in doorpad_moves for move in sublist]

doorpadbot_moves = [doorpadbot.press(target) for target in doorpad_moves]
doorpadbot_moves = [move for sublist in doorpadbot_moves for move in sublist]
doorpadbot_moves = "".join(doorpadbot_moves)

assert doorpadbot_moves == 'v<<A>>^A<A>A<AAv>A^Av<AAA^>A', doorpadbot_moves
rev = doorpadbot.reverse_presses(doorpadbot_moves)
print(rev)
assert rev == doorpad_moves, rev
rev2 = doorpad.reverse_presses(rev)
print(rev2)
assert rev2 == list(sequence), rev2

['<', 'A', '^', 'A', '^', '^', '>', 'A', 'v', 'v', 'v', 'A']
['0', '2', '9', 'A']


In [6]:
doorpad = Keypad(num_keypad, directional_keypad, 'doorpad')
doorpadbot = Keypad(directional_keypad, directional_keypad, 'doorpadbotpad')
doorpadbotbot = Keypad(directional_keypad, directional_keypad, 'doorpadbotbotpad')

sequence = "379A"

doorpad_moves = [doorpad.press(target) for target in sequence]
doorpad_moves = [move for sublist in doorpad_moves for move in sublist]
doorpad_moves = "".join(doorpad_moves)

doorpadbot_moves = [doorpadbot.press(target) for target in doorpad_moves]
doorpadbot_moves = [move for sublist in doorpadbot_moves for move in sublist]
doorpadbot_moves = "".join(doorpadbot_moves)

doorpadbotbot_moves = [doorpadbotbot.press(target) for target in doorpadbot_moves]
doorpadbotbot_moves = [move for sublist in doorpadbotbot_moves for move in sublist]
doorpadbotbot_moves = "".join(doorpadbotbot_moves)

print(doorpadbotbot_moves)
rev = doorpadbotbot.reverse_presses(doorpadbotbot_moves)
print("".join(rev))
rev2 = doorpadbot.reverse_presses(rev)
print("".join(rev2))
rev3 = doorpad.reverse_presses(rev2)
print("".join(rev3))

sequence_num = int(sequence.replace('A', ''))
print(sequence_num)
print(len(doorpadbotbot_moves))
print(f"complexity: {len(doorpadbotbot_moves)*sequence_num}")
assert len(doorpadbotbot_moves)*sequence_num == 24256, len(doorpadbotbot_moves)*sequence_num

v<<A>>^AvA^Av<<A>>^AAv<A<A>>^AAvAA^<A>Av<A^>AA<A>Av<A<A>>^AAA<Av>A^A
<A>A<AAv<AA>>^AvAA^Av<AAA^>A
^A^^<<A>>AvvvA
379A
379
68
complexity: 25772


AssertionError: 25772

In [15]:
def calc_complexity(sequence: list[str]) -> int:
    doorpad = Keypad(num_keypad, directional_keypad, 'doorpad')
    doorpadbot = Keypad(directional_keypad, directional_keypad, 'doorpadbotpad')
    doorpadbotbot = Keypad(directional_keypad, directional_keypad, 'doorpadbotbotpad')
    assert doorpad.current_position == doorpad.child_keypad_map['A'], doorpad.current_position
    assert doorpadbotbot.current_position == doorpadbotbot.child_keypad_map['A'], doorpadbotbot.current_position

    doorpad_moves = [doorpad.press(target) for target in sequence]
    doorpad_moves = [move for sublist in doorpad_moves for move in sublist]

    doorpadbot_moves = [doorpadbot.press(target) for target in doorpad_moves]
    doorpadbot_moves = [move for sublist in doorpadbot_moves for move in sublist]

    doorpadbotbot_moves = [doorpadbotbot.press(target) for target in doorpadbot_moves]
    doorpadbotbot_moves = [move for sublist in doorpadbotbot_moves for move in sublist]
    doorpadbotbot_moves = "".join(doorpadbotbot_moves)

    sequence_num = int(sequence.replace('A', ''))
    return (doorpadbotbot_moves, sequence_num, len(doorpadbotbot_moves)*sequence_num)

def run(sequences: list[str]) -> int:
    complexity_sum = 0
    sequence_nums = []
    move_sets = []

    for sequence in sequences:
        moves, sequence_num, com = calc_complexity(sequence)
        complexity_sum += com
        sequence_nums.append(sequence_num)
        move_sets.append(moves)

    print(f"complexity sum: {complexity_sum}")
    for i, _ in enumerate(sequences):
        print(sequences[i], end=': ')
        print(f"len: {len(move_sets[i])}, ", end=' ')
        print(f"sequence_num: {sequence_nums[i]}, ", end=' ')
        print(f"complexity: {len(move_sets[i])*sequence_nums[i]}")
        print(f"moves: {move_sets[i]}")
        print()

    return complexity_sum

In [16]:
sequences = ["029A","980A","179A","456A","379A"]
complexity_sum = run(sequences)
assert complexity_sum == 126384, complexity_sum

complexity sum: 127900
029A: len: 68,  sequence_num: 29,  complexity: 1972
moves: v<A<AA>>^AvAA^<A>Av<<A>>^AvA^Av<<A>>^AAv<A>A^A<A>Av<A<A>>^AAA<Av>A^A

980A: len: 60,  sequence_num: 980,  complexity: 58800
moves: v<<A>>^AAAvA^Av<A<AA>>^AvAA^<A>Av<A<A>>^AAA<Av>A^Av<A^>A<A>A

179A: len: 68,  sequence_num: 179,  complexity: 12172
moves: v<<A>>^Av<A<A>>^AAvAA^<A>Av<<A>>^AAvA^Av<A^>AA<A>Av<A<A>>^AAA<Av>A^A

456A: len: 64,  sequence_num: 456,  complexity: 29184
moves: v<<A>>^AAv<A<A>>^AAvAA^<A>Av<A^>A<A>Av<A^>A<A>Av<A<A>>^AA<Av>A^A

379A: len: 68,  sequence_num: 379,  complexity: 25772
moves: v<<A>>^AvA^Av<<A>>^AAv<A<A>>^AAvAA^<A>Av<A^>AA<A>Av<A<A>>^AAA<Av>A^A



AssertionError: 127900

In [10]:
sequences = ["670A","974A","638A","319A","508A"]
complexity_sum = run(sequences)

complexity sum: 226518
670A: len: 72,  sequence_num: 670,  complexity: 48240
moves: v<<A>>^AAvA^Av<<A>>^Av<A<A>>^AAvAA^<A>Av<A^>Av<<A>>^AAA<Av>A^Av<A^>A<A>A

974A: len: 72,  sequence_num: 974,  complexity: 70128
moves: v<<A>>^AAAvA^Av<A<AA>>^AAvAA^<A>Av<A<A>>^A<Av>A^Av<A^>AAv<<A>>^AA<Av>A^A

638A: len: 74,  sequence_num: 638,  complexity: 47212
moves: v<<A>>^AAvA^Av<A<A>>^A<Av>A^Av<<A>>^AAv<A<A>>^AvAA^<A>Av<A<A>>^AAAvA^A<A>A

319A: len: 70,  sequence_num: 319,  complexity: 22330
moves: v<<A>>^AvA^Av<A<AA>>^AAvAA^<A>Av<<A>>^AAv<A>A^AA<A>Av<A<A>>^AAA<Av>A^A

508A: len: 76,  sequence_num: 508,  complexity: 38608
moves: v<<A>>^AAv<A<A>>^AvAA^<A>Av<A<A>>^AA<Av>A^Av<<A>>^AAAvA^Av<A<A>>^AAAvA^A<A>A

